# Data loading 
Next cells are used to load the data we need.

In [11]:
import pandas as pd
import numpy as np 
import scipy.sparse as sps
import os
import sys
import matplotlib.pyplot as plt
sys.path.append('..')


In [12]:
#data which contains users (row), items(col) and implicit interaction (data)
dataset = pd.read_csv('data_train.csv')
dataset

,row,col,data
0,0,10080,1.0
1,0,19467,1.0
2,1,2665,1.0
3,1,7494,1.0
4,1,17068,1.0
...,...,...,...
113263,7945,2476,1.0
113264,7945,12319,1.0
113265,7945,21384,1.0
113266,7946,8699,1.0


# Data pre-processing
Pre-processing of data to check for missing users, items ecc...

In [13]:
users = dataset.row
items = dataset.col
data = dataset.data
URM_all = sps.coo_matrix((data, (users, items)))
URM_all = URM_all.tocsr() #fast row access -> fast access to users 
URM_all.shape

(7947, 25975)

In [14]:
ICM_df = pd.read_csv('data_ICM_title_abstract.csv')
ICM_df

,row,col,data
0,0,1185,1.015524
1,0,2507,0.459024
2,0,3534,0.227742
3,0,8766,0.501549
4,0,10862,0.297011
...,...,...,...
490686,25974,12554,0.963016
490687,25974,13003,0.104613
490688,25974,16236,0.118760
490689,25974,18797,0.363301


In [15]:
items = ICM_df.row
features = ICM_df.col
data = ICM_df.data
ICM_all = sps.coo_matrix((data, (items, features)))
ICM_all = ICM_all.tocsr() #fast row access -> fast access to users 
ICM_all.shape

(25975, 20000)

In [16]:
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

#splitting data into train and validation (from the previous training data)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.85)

In [17]:
URM_all

<7947x25975 sparse matrix of type '<class 'numpy.float64'>'
	with 113268 stored elements in Compressed Sparse Row format>

In [18]:
ICM_all

<25975x20000 sparse matrix of type '<class 'numpy.float64'>'
	with 490691 stored elements in Compressed Sparse Row format>

In [19]:
#evaluator for validation (used for hyperparameter tuning)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [48]:
from GraphBased.RP3betaRecommender import RP3betaRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender


recommender_beta_ICM = RP3betaRecommender(ICM_all.T)
recommender_beta_ICM.fit(topK=70, alpha=0.2, beta=0.5, implicit=False)
recommender_beta_ICM.URM_train = URM_train

recommeder_KNNItemCBF = ItemKNNCBFRecommender(URM_train, ICM_all)
recommeder_KNNItemCBF.fit(topK=100, shrink=300, feature_weighting = 'TF-IDF') #Vedere quale normalizzazione ha più senso dal punto di vista teorico


hybridrecommender = ItemKNNSimilarityHybridRecommender(URM_train, recommender_beta_ICM.W_sparse, recommeder_KNNItemCBF.W_sparse)
hybridrecommender.fit(topK=200, alpha = 0.9)

evaluator_validation.evaluateRecommender(hybridrecommender)

RP3betaRecommender: URM Detected 2 (0.01 %) cold users.
RP3betaRecommender: URM Detected 25 (0.10 %) cold items.
ItemKNNCBFRecommender: URM Detected 41 (0.52 %) cold users.
ItemKNNCBFRecommender: URM Detected 2023 (7.79 %) cold items.
Similarity column 25975 ( 100 % ), 3120.46 column/sec, elapsed time 0.14 min
ItemKNNSimilarityHybridRecommender: URM Detected 41 (0.52 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 2023 (7.79 %) cold items.
EvaluatorHoldout: Processed 5010 ( 100.00% ) in 4.34 sec. Users per second: 1155


({10: {'ROC_AUC': 0.10590305104077567,
   'PRECISION': 0.02235528942115744,
   'PRECISION_RECALL_MIN_DEN': 0.0894119697113709,
   'RECALL': 0.08732724729594653,
   'MAP': 0.03826823837208416,
   'MRR': 0.0745264233437885,
   'NDCG': 0.058562177079571724,
   'F1': 0.03559775231473912,
   'HIT_RATE': 0.22355289421157684,
   'ARHR': 0.08245421854703286,
   'NOVELTY': 0.005352647026659024,
   'AVERAGE_POPULARITY': 0.0170178248154854,
   'DIVERSITY_MEAN_INTER_LIST': 0.998171514826207,
   'DIVERSITY_HERFINDAHL': 0.9997972278994904,
   'COVERAGE_ITEM': 0.5972666025024062,
   'COVERAGE_ITEM_CORRECT': 0.03869104908565929,
   'COVERAGE_USER': 0.6304265760664401,
   'COVERAGE_USER_CORRECT': 0.11413111866112999,
   'DIVERSITY_GINI': 0.3094748476560891,
   'SHANNON_ENTROPY': 13.228906407536241}},
 'CUTOFF: 10 - ROC_AUC: 0.1059031, PRECISION: 0.0223553, PRECISION_RECALL_MIN_DEN: 0.0894120, RECALL: 0.0873272, MAP: 0.0382682, MRR: 0.0745264, NDCG: 0.0585622, F1: 0.0355978, HIT_RATE: 0.2235529, ARHR: 0

In [ ]:
test_users = pd.read_csv('data_target_users_test.csv')
test_users

In [ ]:
#0.039002456651731125
#topk 70, alpha0.2, beta0.4

recommender = ItemKNNCBFRecommender(URM_all, ICM_all)
recommender.fit(topK=600, shrink=200, feature_weighting = 'TF-IDF')

user_id = test_users['user_id']
recommendations = recommender.recommend(user_id,cutoff = 10)


In [ ]:
for index in range(len(recommendations)):
    #print(element)
    recommendations[index]=np.array(recommendations[index])
    #print(type(element))
print(len(recommendations))

In [ ]:
test_users['item_list']= recommendations
#test_users['item_list'] =  test_users['item_list'].apply(lambda x: x.replace('[','').replace(']','')) 
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])

#convert the string columns to int
#test_users['item_list'] = test_users['item_list'].astype(int)
test_users
test_users.to_csv('submission.csv', index=False)
